In [1]:
import pandas as pd

In [27]:
sp100 = pd.read_csv('data/sp100_data.csv')
tickers = pd.read_csv('data/tickers_list.csv')

print(f'The S&P 100 dataset has {sp100.shape[0]} rows and {sp100.shape[1]} columns')
print(f'There is {tickers.shape[0]} companies collected from the S&P 100 index')

The S&P 100 dataset has 1006 rows and 506 columns
There is 101 companies collected from the S&P 100 index


In [28]:
tickers.head()

,0
0,AAPL
1,ABBV
2,ABT
3,ACN
4,ADBE


In [14]:
tickers.shape

(101, 1)

In [31]:
tickers['0'][0]

'AAPL'